# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import matplotlib.pyplot as plt
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import pickle
import re
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\as572v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\as572v\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\as572v\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_disaster', con = engine)
X = df['message']
y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight = 'balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        class_weight='balanced',
                                                                        criterion='gini',
                                                                        max_depth=None,
                                                            

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def report(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [9]:
report(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.75      0.24      0.36      1576
           1       0.80      0.97      0.88      4936
           2       0.25      0.02      0.04        42

    accuracy                           0.79      6554
   macro avg       0.60      0.41      0.43      6554
weighted avg       0.78      0.79      0.75      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.89      0.44      0.59      1109

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\as572v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6268
           1       0.00      0.00      0.00       286

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.96      0.93      6554

buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6209
           1       0.82      0.03      0.05       345

    accuracy                           0.95      6554
   macro avg       0.88      0.51      0.51      6554
weighted avg       0.94      0.95      0.92      6554

electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6419
           1       1.00      0.01      0.01       135

    accuracy                           0.98      6554
   macro avg       0.99      0.50      0.50      6554
weighted avg       0.98      0.98      0.97      655

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [20, 50], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)
report(cv, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.69      0.32      0.44      1576
           1       0.81      0.95      0.88      4936
           2       0.50      0.21      0.30        42

    accuracy                           0.80      6554
   macro avg       0.67      0.50      0.54      6554
weighted avg       0.78      0.80      0.77      6554

request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5445
           1       0.84      0.54      0.65      1109

    accuracy                           0.90      6554
   macro avg       0.87      0.76      0.80      6554
weighted avg       0.90      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\as572v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6339
           1       0.00      0.00      0.00       215

    accuracy                           0.97      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.94      0.97      0.95      6554

death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6279
           1       0.87      0.10      0.18       275

    accuracy                           0.96      6554
   macro avg       0.92      0.55      0.58      6554
weighted avg       0.96      0.96      0.95      6554

other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5703
           1       0.62      0.04      0.08       851

    accuracy                           0.87      6554
   macro avg       0.75      0.52      0.51      6554
weighted avg       0.84      0.87      0.82      6554

inf

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
improve_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [15]:
improve_pipeline.fit(X_train, y_train)
report(improve_pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.62      0.01      0.01      1576
           1       0.75      1.00      0.86      4936
           2       0.43      0.07      0.12        42

    accuracy                           0.75      6554
   macro avg       0.60      0.36      0.33      6554
weighted avg       0.72      0.75      0.65      6554

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5445
           1       0.00      0.00      0.00      1109

    accuracy                           0.83      6554
   macro avg       0.42      0.50      0.45      6554
weighted avg       0.69      0.83      0.75      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\as572v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6279
           1       0.00      0.00      0.00       275

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554

other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5703
           1       0.00      0.00      0.00       851

    accuracy                           0.87      6554
   macro avg       0.44      0.50      0.47      6554
weighted avg       0.76      0.87      0.81      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      6098
           1       0.00      0.00      0.00       456

    accuracy                           0.93      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.87      0.93      0.

In [16]:
improve_parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2]}

In [17]:
improve_cv = GridSearchCV(improve_pipeline, param_grid = improve_parameters)
improve_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [18]:
report(improve_cv, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.62      0.01      0.01      1576
           1       0.75      1.00      0.86      4936
           2       0.43      0.07      0.12        42

    accuracy                           0.75      6554
   macro avg       0.60      0.36      0.33      6554
weighted avg       0.72      0.75      0.65      6554

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5445
           1       0.00      0.00      0.00      1109

    accuracy                           0.83      6554
   macro avg       0.42      0.50      0.45      6554
weighted avg       0.69      0.83      0.75      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\as572v\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



clothing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6449
           1       0.00      0.00      0.00       105

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.98      0.98      6554

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6418
           1       0.00      0.00      0.00       136

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.96      0.98      0.97      6554

missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98

### 9. Export your model as a pickle file

In [19]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(improve_cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.